# Tutorial 2.1: One-Body Potential Energy Surface

In this section, we construct, from scratch, a 1-body MB-nrg-style PEF for CH3-NH2.

## 2.1.0 Imports, Initialization, Filepaths

Let's begin by doing some setup. 

First, lets import everything we will use in this notebook.

In [8]:
import os

We will need a place to store files as we construct our PES, including input files, temporary files, and output files. Feel free to set `monomer_CH3NH2_workdir` to whichever directory you would like to use:

In [12]:
monomer_CH3NH2_workdir = "HCN_1B_PEF"
if not os.path.exists(HCN_workdir):
    os.makedirs(HCN_workdir)

Inside this directory, we will need places to put lots of files, lets define these paths here:

In [15]:
monomer_CH3_NH2_paths = {
    "workdir": monomer_CH3NH2_workdir
    "definition": os.path.join(monomer_CH3NH2_workdir, "monomer.def")
}

More information on what each of these files contains is provided as they are used in the following sections.

Finally, we will initialize a few objects that we will use farther down in the notebook:

In [ ]:
job_runner = SLURMJobRunner(
    ...
)

## 2.1.1 Input Setup

Let's define our monomer's definition. The monomer definition file contains all the non-structure-specific information about the monomer we will study. This includes atom elements, atom types (or "symmetries"), and bonding topology - but not atom coordinates, since atom coordinates are structure-specific. We will pass this definition to various functions in MB-Fit that need some or all of this information.

In [ ]:
monomer_CH3NH2_definition = BasicFragmentDefinition()
CA  = monomer_CH3NH2_definition.new_atom("C", "A")
HB1 = monomer_CH3NH2_definition.new_atom("H", "B")
HB2 = monomer_CH3NH2_definition.new_atom("H", "B")
HB3 = monomer_CH3NH2_definition.new_atom("H", "B")
NC  = monomer_CH3NH2_definition.new_atom("N", "C")
HD1 = monomer_CH3NH2_definition.new_atom("H", "D")
HD2 = monomer_CH3NH2_definition.new_atom("H", "D")
monomer_CH3NH2_definition.add_bond(CA, HB1, BondType.SINGLE)
monomer_CH3NH2_definition.add_bond(CA, HB2, BondType.SINGLE)
monomer_CH3NH2_definition.add_bond(CA, HB3, BondType.SINGLE)
monomer_CH3NH2_definition.add_bond(CA, NC, BondType.SINGLE)
monomer_CH3NH2_definition.add_bond(NC, HD1, BondType.SINGLE)
monomer_CH3NH2_definition.add_bond(NC, HD2, BondType.SINGLE)

definition_writer(monomer_CH3_NH2_paths["definition"], monomer_CH3NH2_definition)

Next, we need a reasonable initial guess for the structure of our monomer. The guess need only be good enough that it will covnerge to the actual minimum structure when we perform a minimization with DFT. MB-Fit contains the class `VESPRInitializer`, which will make a reasonable guess about the structure of a molecule based on its bonding topology. Let's use `VESPRInitializer` to create an initial structure file:

In [ ]:
definition = definition_reader(monomer_CH3NH2_definition)
system_initializer = VESPRInitializer()
initial_structure = system_initializer(definition)
xyz_writer(monomer_CH3NH2_initial_structure, initial_structure)

Next, we create the settings file. The settings file contains a few pieces of information that will be used down the road to construct our 1-body PEF.

## II.1.2 Geometry Optimization

That guess is looking a little rough, lets optimize it using electronic structure theory. While the PEF will be developed at the CCSD(T) level of theory, its not important that the geometry optimization be at the same level of theory as the reference energies will be calculated at, so lets use something cheaper like a high-quality DFT functional.

In [ ]:
job = job_runner.queue_job(
"""
calculator = QChemCalculator("wb97m-v", "aug-cc-pvTZ")
initial_structure = xyz_reader(monomer_CH3NH2_initial_structure)
optimized_structure, optimized_energy, optimization_log_path = calculator.energy(initial_structure)
xyz_writer(monomer_CH3NH2_optimized_structure, optimized_structure)
"""
)

job.wait_until_done()

## II.1.? $V_{phys}$ Parametrization

Next, we will parameterize the "physical baseline" ($V_{phys}$) of the PEF.

First, we will calculate the dispersion coefficients ($C_6$), dispersion damping parameters ($d_6$), atomic polarizabilities ($p$) and polarizability factors ($a$), and atomic radii ($\rho$)

In [25]:
job = job_runner.queue_job(
"""
    calculator = QChemCalculator("wb97m-v", "aug-cc-pvTZ")
    optimized_structure = xyz_reader(monomer_CH3NH2_optimized_structure)
    C6, d6 = mbfit.parametrization.classical.calculate_dispersion_parameters(optimized_structure, calculator)
    disp_parameters_writer(..., (C6, d6))
"""
)

job.wait_until_running()
print("Job has made it though queue and is running!")

job.wait_until_done()
print("Job completed!")

Next, we will calculate the atomic partial charges ($q_i$).

In [26]:
job = job_runner.queue_job(
"""
"""
)

job.wait_until_running()
print("Job has made it though queue and is running!")

job.wait_until_done()
print("Job completed!")

Now, lets take a look at how well these classical parameters describe this system. While we are focusing on the monomer here, we will use a basic scan of the dimer to seen how well the $V_{phys}$ part of our PEF is looking.

## II.1.c Normal Modes Configuration Generation

Now, we will generate the first batch of configurations for our training set. This batch will come from randomly sampling the vibrational normal modes of methylamine.

First, we need to calculate the vibrational modes of the optimized geometry:

In [ ]:
job = job_runner.queue_job(
"""
optimized_structure = xyz_reader(definition, monomer_CH3NH2_optimized_structure)
vibrational_modes, vibrational_modes_log_path = calculator.normal_modes(optimized_structure)
vibrational_modes_writer(monomer_CH3NH2_vibrational_modes, vibrational_modes)
"""
)

job.wait_until_running()
print("Job has made it though queue and is running!")

job.wait_until_done()
print("Job completed!")


Note: the seed here determines which configurations get generated. The same seed will always produce the same configurations. In a later step, we "cheat" and have pre-performed the CCSD(T) calculations for these structures, so please don't change the seed, because then the energies we give you wont match the structures that get generated!

In [ ]:
vibrational_modes_configuration_generator = NormalModesConfigurationGenerator(optimized_structure, vibrational_modes)
vibrational_modes_configs = vibrational_modes_configuration_generator.generate_configs(1000)

## II.1.? Parallel-Bias Metadynamics Configuration Generation

Let's generate the second set of configurations for our training set, those from metadynamics configurations.

In [ ]:
job = job_runner.queue_job(
f"""
optimized_structure = xyz_reader(definition, monomer_CH3NH2_optimized_structure)
mbfit....generate_metadynamis_configurations(...)
mbfit.perform_fps_sampling(...)
"""
)

job.wait_until_running()
print("Job has made it though queue and is running!")

job.wait_until_done()
print("Job completed!")

## II.1.? Energy Calculation

Next, we need to perform the electronic structure calculations with our chosen reference level of theory. We will use CCSD(T). Because this can be quite expensive and this tutorial is menat to be something that can be finished in a few hours, we have precomputed the energies for you. So we will just copy them over:

In [ ]:
shutil.copyfile("...", monomer_CH3NH2_file_paths.training_set_file)

If you did want to run the energy calculations your self, this is how you could do it:

In [2]:
...

Ellipsis

## II.1.? Polynomial Generation

## II.1.? Polynomial Parametrization

## II.1.? Implementation in MBX

## II.1.? Test MBX Implementation

## II.1.? Run a Gas-Phase Monomer Simulation